In [1]:
import torch
import numpy as np
import json

In [2]:
batch_size, seq_len, word_len, char_dim = 10, 11, 16, 20

In [20]:
char_emb = torch.randn(batch_size, seq_len, word_len, char_dim)
char_emb, _ = torch.max(char_emb, dim=2, keepdim=True)
char_emb = char_emb.squeeze(dim=2)
char_emb.size()

torch.Size([10, 11, 20])

In [2]:
import torch.nn as nn
import torch.nn.functional as F

In [49]:
conv = nn.Conv1d(500, 128, 7, padding = 3)

In [51]:
x = torch.randn(10, 100, 500)


In [52]:
conv(x.permute(0,2,1)).size()

torch.Size([10, 128, 100])

In [55]:
import util
word_vectors = util.torch_from_json("data/word_emb.json")
word_vectors

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0828,  0.6720, -0.1499,  ..., -0.1918, -0.3785, -0.0659],
        ...,
        [ 0.3990, -0.0185,  0.3715,  ...,  0.0370, -0.0975, -0.6882],
        [-0.0652, -0.0192, -0.8155,  ...,  0.9371, -0.4975, -1.1756],
        [ 0.5272, -1.0988, -0.3210,  ..., -0.0951, -0.1996, -1.2129]])

In [56]:
word_vectors.size()

torch.Size([88714, 300])

In [59]:
torch.autograd.Variable(torch.zeros(1,4), requires_grad=False)

tensor([[0., 0., 0., 0.]])

## 字典反查

In [61]:
dict = {'a':0, 'b':0, 'c':1}

In [62]:
{value:key for key,value in dict.items()}

{0: 'b', 1: 'c'}

## 如果要保留重复的值进list？ 有点意思

In [71]:
new_dict = {}
for key, value in dict.items():
    if new_dict.get(value, False):
        new_dict[value] = new_dict[value].append(key) if isinstance(new_dict[value],list) else [new_dict[value], key]
    else:
        new_dict[value] = key

In [72]:
new_dict

{0: ['a', 'b'], 1: 'c'}

# 08/11维度检查

In [77]:
%load_ext autoreload
%autoreload 2
from layers_QANet import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
posenc = PositionalEncoding(500, 0, 400)
cnn_init = CNN(input_dim=500,
                            kernel_size=7,
                            padding=3,
                            filters=128)
cnn = CNN(input_dim=128,
                       kernel_size=7,
                       padding=3,
                       filters=128)
layernorm = LayerNorm(128)
att = MultiHeadedAttention(h=8,
                                        d_model=128,
                                        dropout=0)
ffn = FFN(input_dim=128,
                       output_dim=128,
                       dropout=0)

In [38]:
input = torch.zeros(3, 16, 500)

In [51]:
x = posenc(input)
x = cnn_init(x)
print(x.size())
for i in range(4):
    x += cnn(layernorm(x))

q = layernorm(x)
print(q.size())
x += att(q, None)
x += ffn(layernorm(x))

x.size()

torch.Size([3, 16, 128])
torch.Size([3, 16, 128])


torch.Size([3, 16, 128])

In [53]:
list(zip([1,1,1,1], [2,2,2]))

[(1, 2), (1, 2), (1, 2)]

In [60]:
conv = nn.Conv1d(
            in_channels=500,
            out_channels=128,
            kernel_size=7,
            padding=3,
            bias=True,
        )
x = conv(input.permute(0,2,1)).permute(0,2,1)

In [62]:
conv = nn.Conv1d(
            in_channels=128,
            out_channels=128,
            kernel_size=7,
            padding=3,
            bias=True,
        )
conv(x.permute(0,2,1)).permute(0,2,1).size()

torch.Size([3, 16, 128])

In [68]:
torch.randn(64,257).unsqueeze(1).unsqueeze(1).size()

torch.Size([64, 1, 1, 257])

In [70]:
torch.Tensor.masked_fill_?

Docstring:
masked_fill_(mask, value)

Fills elements of :attr:`self` tensor with :attr:`value` where :attr:`mask` is
True. The shape of :attr:`mask` must be
:ref:`broadcastable <broadcasting-semantics>` with the shape of the underlying
tensor.

Args:
    mask (BoolTensor): the boolean mask
    value (float): the value to fill in with
Type:      method_descriptor


In [76]:
scores = torch.randn(64, 8, 20, 20)
mask = torch.ones(64, 20)
mask = mask.unsqueeze(1).unsqueeze(1)
scores.masked_fill(mask==0, -1e9)

5e-02, -1.0837e-01],
          [-2.0799e-01, -2.8529e-01, -2.3916e-01,  ...,  1.3496e+00,
            1.5313e-01,  4.5822e-01]]],


        [[[-2.2337e-01, -8.0660e-01, -2.4637e-02,  ..., -1.2298e+00,
           -2.9158e-01, -6.2693e-01],
          [ 1.7014e+00,  2.0341e+00, -2.8929e+00,  ..., -9.5293e-01,
            1.0984e+00,  5.4336e-01],
          [-1.1239e+00, -1.2064e+00,  2.9586e-01,  ..., -4.4473e-02,
           -5.2360e-01,  4.3700e-01],
          ...,
          [ 2.6286e-01,  1.0938e+00,  8.5633e-02,  ...,  3.4203e-01,
            1.8694e-01, -5.0930e-01],
          [ 1.9334e-01, -1.6825e-01,  1.3490e+00,  ...,  1.0281e+00,
           -6.7471e-02,  4.3761e-01],
          [ 8.7041e-01, -1.0954e+00,  2.7368e-01,  ...,  3.4801e-01,
           -1.3121e+00,  6.4257e-01]],

         [[-5.4967e-01,  6.5172e-01, -1.0226e-01,  ...,  1.4907e+00,
            5.3949e-02, -1.5420e+00],
          [ 5.5628e-01, -2.6444e-01, -9.1508e-01,  ...,  1.6519e+00,
           -6.6285e-01, -2.8100e-